In [12]:
import pandas as pd
import pickle
file = open("df.pickle","rb")
df = pickle.load(file)
num_csv = int(max(df['Count']))+1
df.shape

(3431, 6)

In [13]:
def clean_df(df):
    df = df.fillna(0)
    df = df.reindex(sorted(df.columns), axis=1)
    df.rename(columns=dict(zip(df.columns,[str(int(c)) if not c[0].isalpha() else c for c in df.columns])),inplace=True)
    return df


df_aantal = df.pivot(index='Gemeente', columns='Count', values='Aantal').reset_index()
df_aantal = clean_df(df_aantal)


df_relative = df.pivot(index='Gemeente', columns='Count', values='Aantal per 100000 inwoners').reset_index()
df_relative = clean_df(df_relative)

In [14]:
# make a growth df
df_growth = df_aantal
temp_columns = df_growth.columns
import numpy as np
def calc_growth(line):
    # line = df_aantal[df_aantal['Gemeente']=='Tilburg'].values[0]
    # growth defined as today/yesterday -1
    # print(line)
    growth = [y/x-1 if x>4 else 0 for x,y in zip(line[:-1],line[1:-1])]
    # moving average over the growth
    smoothed_growth = [(x1+x2+x3+x4)/3 for x1,x2,x3,x4 in zip(growth,growth[1:],growth[2:],growth[3:])]
    # growth defined as today/4 days ago -1
    alternative_growth = [y/x-1 if x>4 else 0 for x,y in zip(line[:-1],line[3:-1])]
    result = [0]*(len(line)-len(alternative_growth)-1) + alternative_growth + [line[-1]]
    # print(len(line), len(growth), len(smoothed_growth),len(alternative_growth), len(result))
    return result

df_growth = pd.DataFrame.from_records(df_growth.apply(calc_growth,axis=1))
df_growth.rename(columns=dict(zip(df_growth.columns,temp_columns)),inplace=True)

In [15]:
import altair as alt
import pandas as pd
from vega_datasets import data
url1 = 'https://cartomap.github.io/nl/rd/gemeente_2020.topojson'
url2 = 'https://cartomap.github.io/nl/wgs84/gemeente_2020.topojson' 

gemeentes = alt.topo_feature(url2, 'gemeente_2020')

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

columns = [str(day) for day in range(0, num_csv)]

slider = alt.binding_range(min=0, max=num_csv-1, step=1)
select_year = alt.selection_single(name="day", fields=['day'],
                                   bind=slider, init={'day': num_csv-1})
def serve_chart(df, legend):
    chart = alt.Chart(gemeentes).mark_geoshape(
        stroke='black',
        strokeWidth=0.05
    ).transform_lookup(
        lookup='properties.statnaam',
        from_=alt.LookupData(df, 'Gemeente', columns),
        default = '0'
    ).transform_fold(
        columns, as_=['day', legend]
    ).transform_calculate(
        day='parseInt(datum.day)',
        legend='isValid(datum.'+legend+') ? datum.'+legend+' : -1'  
    ).encode(
        tooltip=['properties.statnaam:N','18:Q'],
        color = alt.condition(
            'datum.'+legend+' > 0',
            alt.Color(legend+':Q', scale=alt.Scale(scheme='spectral', type='symlog',domain=[max(df[str(num_csv-1)]),0])),
            alt.value('#dbe9f6')
        )).add_selection(
        select_year
    ).properties(
        width=300,
        height=400
    ).transform_filter(
        select_year
    )    
    return chart


In [16]:
alt.hconcat(
    serve_chart(df_aantal, 'absoluut'), serve_chart(df_relative, 'relatief'),serve_chart(df_growth, 'groei')
).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
)

alt.HConcatChart(...)

In [17]:
# inspect individual cities
x = list(df_aantal[df_aantal['Gemeente']=="Amsterdam"].values[0][:-1])
y = list(range(len(x)))
df = pd.DataFrame(x,y).reset_index()
print(df.columns)
df.rename(columns=dict(zip(df.columns,[str(c) for c in df.columns])),inplace=True)

alt.Chart(df).mark_point().encode(
    x='index:Q',
    y='0:Q',
)

Index(['index', 0], dtype='object')


alt.Chart(...)